In [10]:
SCOPETYPE = 'CWNANO'
PLATFORM = 'CWNANO'
CRYPTO_TARGET='TINYAES128C' 
SS_VER='SS_VER_2_1'

In [11]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.
INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.
INFO: Found ChipWhisperer😍


In [12]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd ../../../firmware/mcu/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 -j

Building for platform CWNANO with CRYPTO_TARGET=TINYAES128C
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
Blank crypto options, building for AES128
arm-none-eabi-gcc (15:13.2.rel1-2) 13.2.1 20231009
Copyright (C) 2023 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

mkdir -p objdir-CWNANO 
.
Welcome to another exciting ChipWhisperer target build!!
.
.
.
.
.
Compiling:
Compiling:
.
.
Compiling:
Compiling:
Compiling:
-en     .././simpleserial/simpleserial.c ...
-en     simpleserial-aes.c ...
Compiling:
Compiling:
-en     .././hal//stm32f0_nano/stm32f0_hal_nano.c ...
-en     .././hal/hal.c ...
-en     .././hal//stm32f0/stm32f0_hal_lowlevel.c ...
.
-en     .././crypto/tiny-AES128-C/aes.c ...
-en     .././crypto/aes-independant.c ...
Assembling: .././hal//stm32f0/stm32f0_startup.S
arm-none-eabi-gcc -c -mcpu=cortex-m0 -I. -x assembler-with-cpp -mthumb -mfl

/usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/bin/ld: /usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/lib/thumb/v6-m/nofp/libg_nano.a(libc_a-closer.o): in function `_close_r':
/build/newlib-38V0JC/newlib-4.4.0.20231231/build_nano/arm-none-eabi/thumb/v6-m/nofp/newlib/../../../../../../newlib/libc/reent/closer.c:47:(.text+0xc): warning: _close is not implemented and will always fail
/usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/bin/ld: /usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/lib/thumb/v6-m/nofp/libg_nano.a(libc_a-lseekr.o): in function `_lseek_r':
/build/newlib-38V0JC/newlib-4.4.0.20231231/build_nano/arm-none-eabi/thumb/v6-m/nofp/newlib/../../../../../../newlib/libc/reent/lseekr.c:49:(.text+0x10): warning: _lseek is not implemented and will always fail
/usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/bin/ld: /usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/lib/thumb/v6-m/nofp/libg_nano.a(libc_a-readr.o): in function `_read_r

/usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/bin/ld: /usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/lib/thumb/v6-m/nofp/libg_nano.a(libc_a-closer.o): note: the message above does not take linker garbage collection into account
/usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/bin/ld: /usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/lib/thumb/v6-m/nofp/libg_nano.a(libc_a-lseekr.o): note: the message above does not take linker garbage collection into account
/usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/bin/ld: /usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/lib/thumb/v6-m/nofp/libg_nano.a(libc_a-readr.o): note: the message above does not take linker garbage collection into account
/usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/bin/ld: /usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/lib/thumb/v6-m/nofp/libg_nano.a(libc_a-writer.o): note: the message above does not take linker garbage collection into account
-e Do

In [13]:
cw.program_target(scope, prog, "../../../firmware/mcu/simpleserial-aes/simpleserial-aes-{}.hex".format(PLATFORM))

Detected known STMF32: STM32F03xx4/03xx6
Extended erase (0x44), this can take ten seconds or more
Attempting to program 6051 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 6051 bytes


In [14]:
#Testing different capture methods, to see if it has any effect

In [15]:
#FROM LAB3_3
from tqdm.notebook import trange
import numpy as np
import time

ktp = cw.ktp.Basic()
trace_array = []
textin_array = []

key, text = ktp.next()

target.set_key(key)

N = 1
for i in trange(N, desc='Capturing traces'):
    scope.arm()
    
    target.simpleserial_write('p', text)
    
    ret = scope.capture()
    if ret:
        print("Target timed out!")
        continue
    
    response = target.simpleserial_read('r', 16)
    
    trace_array.append(scope.get_last_trace())
    textin_array.append(text)
    
    key, text = ktp.next() 
    

Capturing traces:   0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
cw.plot(trace_array[0])


:Curve   [x]   (y)

In [18]:
print(textin_array[0])

CWbytearray(b'a5 25 aa 85 6e 9f f0 f7 c5 16 90 1a 9f a2 b9 89')


In [19]:
#testing with just capture trace method
trace1_array=[]
textin1_array=[]
trace = cw.capture_trace(scope, target, text, key, as_int = True)
trace2 = cw.capture_trace(scope, target, text, key, as_int = True)
trace1_array = trace.wave
textin1_array = trace.textin

In [20]:
cw.plot(trace.wave)*cw.plot(trace2.wave)
#print(textin1_array)

:Overlay
   .Curve.I  :Curve   [x]   (y)
   .Curve.II :Curve   [x]   (y)

In [21]:
!pip install h5py


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
import h5py
import numpy as np
import time
import os
from tqdm.notebook import tnrange
from binascii import hexlify
import matplotlib.pyplot as plt

NUM_TRACES = 4096
project_name = "attack_set"
implementation = os.path.basename('../../../hardware/victims/firmware/simpleserial-aes')
path = f"traces/{implementation}-{NUM_TRACES}-diff{''}.hdf5"


f = h5py.File(project_name+".hdf5", 'w')

f.attrs["trace_type"] = "profile"
f.attrs["data_type"] = "plaintexts different"
f.attrs["implementation"] = implementation
f.attrs["adc_samples"] = scope.adc.samples

trace_set = f.create_dataset("trace", (NUM_TRACES, scope.adc.samples), dtype=np.int16)
plaintext_set = f.create_dataset("data", (NUM_TRACES, 16), dtype=np.uint8)
key_set = f.create_dataset("key", (NUM_TRACES, 16), dtype=np.uint8)

if "mask" in implementation:
    mask_group = f.create_group("mask")
    keymask_set = mask_group.create_dataset("key", (NUM_TRACES,2), dtype=np.uint32)
    datamask_set = mask_group.create_dataset("data", (NUM_TRACES,), dtype=np.uint32)


In [24]:
#creating a loop for capturing traces
NUM_KEYS = 4096
NUM_PT_PER_KEY = 1 
ktp = cw.ktp.Basic()

#We will use capture trace, and scale the traces later
for i in tnrange(NUM_TRACES, desc = "Capturing traces"):
    #changes the key and text every run
    key, pt = ktp.next()
    trace = None
    while trace is None:
        
        #print("pt",pt)
        #print("key",key)
        trace = cw.capture_trace(scope, target, pt, key, as_int=True)
        #checking key
    
    #if cw.bytearray(key_set[key_idx]) != trace.key:
     #   print(f"error, wrong key{key_set[key_idx]} does not match {trace.key[:16]}")
    if pt == trace.textin:
        plaintext_set[i] = np.array(trace.textin[:16])
    elif pt != trace.textin:
        print(f"pt does not match, in {pt}, out {trace.textin}")
        break;
    
    if key == trace.key:
        key_set[i] = np.array(trace.key[:16])
    elif key != trace.key:
        print("key does not match")
        break;  
    
    trace_set[i] = trace.wave

Capturing traces:   0%|          | 0/4096 [00:00<?, ?it/s]